In [ ]:
import keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, BatchNormalization, Dense, Flatten, Activation
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
import pandas as pd
import numpy as np
import pydot_ng
import graphviz
import os
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import pyplot
from matplotlib.pyplot import *
!pip install scikit-plot
import scikitplot as skplt
from sklearn.metrics import classification_report

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
"""
    num_classes = Defines the number of classes we have to predict which are namely 
    Angry, Fear, Happy, Neutral, Surprise, Neutral and Disgust.
    
    From the exploratory Data Analysis we know that The Dimensions of the image are:
    Image Height = 48 pixels
    Image Width = 48 pixels
    Number of channels = 1 because it is a grayscale image.
    
    We will consider a batch size for the training of the image augmentation.

"""
num_classes = 7
batch_size = 32
epochs = 100
img_height = 48
img_width = 48

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/fer/fer2013.csv')
data.head()

In [ ]:
"""
    Reference: https://github.com/oarriaga/face_classification
    We will convert the pixels to list in this method. 
    We split the data by spaces and then take them as arrays and 
    reshape into 48, 48 shape. Then we expand the dimensions 
    and then convert the labels to categorical matrix.

"""

pixels = data['pixels'].tolist() 
faces = []

for pixel_sequence in pixels:
    face = [int(pixel) for pixel in pixel_sequence.split(' ')]
    face = np.asarray(face).reshape(img_height, img_width) 
    faces.append(face.astype('float32'))

faces = np.asarray(faces)
faces = np.expand_dims(faces, -1)

emotions = pd.get_dummies(data['emotion']).values
len(emotions)

In [ ]:
emotions

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# assume data is a pandas DataFrame with 'emotion' column containing emotion labels
emotion_labels = {0:'anger', 1:'disgust', 2:'fear', 3:'happiness', 4: 'sadness', 5: 'surprise', 6: 'neutral'}

# create a figure and axis object
fig, ax = plt.subplots()

# get the counts of each emotion label and sort by index
counts = data['emotion'].value_counts().sort_index()

# define colors for each emotion label
colors = ['tab:red', 'tab:green', 'tab:orange', 'tab:blue', 'tab:purple', 'tab:brown', 'tab:gray']

# create bar plot with counts and colors
ax.bar(emotion_labels.values(), counts, color=colors)

# add labels and title
ax.set_xlabel('Emotion')
ax.set_ylabel('Count')
ax.set_title('Histogram of Emotion Labels')

# add count labels above each bar
for i, v in enumerate(counts):
    ax.text(i, v+10, str(v), ha='center', fontsize=10)

# rotate x-axis tick labels by 45 degrees
plt.xticks(rotation=45)

# display the plot
plt.show()


In [ ]:
from sklearn.model_selection import train_test_split
#using stratify to ensure that class distribution is approximately the same in train and tets set

X_train, X_test, y_train, y_test = train_test_split(faces, emotions, test_size=0.1, random_state=42,stratify=emotions)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=41 )

In [ ]:
X_train = X_train / 255.
X_valid = X_valid / 255.
X_test = X_test / 255.


In [ ]:
X_train.shape,y_train.shape, X_valid.shape,  y_valid.shape , X_test.shape , y_test.shape

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Define the class labels
class_labels =emotion_labels


# Define the colors to use for each set
set_colors = ['blue', 'green', 'red']

# Define the sets of labels to plot
label_sets = [y_train, y_valid, y_test]
set_names = ['Training Set', 'Validation Set', 'Test Set']

# Get the counts for each set and class
counts = [np.sum(labels, axis=0) for labels in label_sets]

# Create the subplot
fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(12, 4))

# Loop over the sets and plot the histograms
for i, ax in enumerate(axs.flat):
    # Create the histogram plot
    bars = ax.bar(class_labels.values(), counts[i], color=set_colors[i])

    # Set the x-axis tick labels to be tilted by 45 degrees
    plt.setp(ax.get_xticklabels(), rotation=45, ha='right')

    # Add the count number on top of each bar
    for bar in bars:
        height = bar.get_height()
        ax.annotate('{:.0f}'.format(height),
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),
                    textcoords="offset points",
                    ha='center', va='bottom')

    ax.set_xlabel('Emotion')
    ax.set_ylabel('Count')
    ax.set_title(set_names[i])

# Set the overall title of the plot
fig.suptitle('Distribution of Emotions in Different Sets')

# # Set the background color of the subplots to white
# plt.rcParams['axes.facecolor'] = 'white'

# # Adjust the spacing between the subplots
# plt.subplots_adjust(wspace=0.3)

# Show the plot
plt.show()


In [ ]:
def build_net(optim):
    img_input = Input(shape=(img_height, img_width, 1))

    Conv1 = Conv2D(filters=32, kernel_size=(3,3), padding='same',
               kernel_initializer='he_normal', activation='relu')(img_input)
    B1 = BatchNormalization()(Conv1)
    M1 = MaxPooling2D(pool_size=(2,2))(B1)
    D1 = Dropout(0.5)(M1)

    Conv2 = Conv2D(filters=64, kernel_size=(3,3), padding='same',
               kernel_initializer='he_normal', activation='relu')(D1)
    B2 = BatchNormalization()(Conv2)
    M2 = MaxPooling2D(pool_size=(2,2))(B2)
    D2 = Dropout(0.5)(M2)

    Conv3 = Conv2D(filters=128, kernel_size=(3,3), padding='same',
               kernel_initializer='he_normal', activation='relu')(D2)
    B3 = BatchNormalization()(Conv3)
    M3 = MaxPooling2D(pool_size=(2,2))(B3)
    D3 = Dropout(0.5)(M3)

    x = Flatten()(D3)
    DN1 = Dense(64, activation='relu', kernel_initializer='he_normal')(x)
    B4 = BatchNormalization()(DN1)
    D4 = Dropout(0.5)(B4)

    output = Dense(num_classes, activation='softmax', kernel_initializer='he_normal')(D4)

    model = Model(img_input, output)

    model.compile(
        loss='categorical_crossentropy',
        optimizer=optim,
        metrics=['accuracy']
    )

    model.summary()

    return model

In [ ]:
model = build_net(Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7))
model.summary()


In [ ]:
predict_x = model.predict(X_test)
classes_x = np.argmax(predict_x, axis=1)

In [ ]:
predict_x

In [ ]:
classes_x

In [ ]:
from keras.utils import plot_model
plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)

In [ ]:
from tensorflow import keras
from keras.utils.vis_utils import plot_model
from keras.utils import np_utils

keras.utils.plot_model(model, to_file='model1.png', show_layer_names=True,rankdir='LR',dpi=90)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import EarlyStopping

tensorboard = TensorBoard(log_dir='logs1')
lr_reducer = ReduceLROnPlateau(monitor='val_accuracy', factor=0.9, patience=3, verbose=1)
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=8, verbose=1, mode='auto')
checkpointer = ModelCheckpoint("emotions1.h5", monitor='val_accuracy', verbose=1, save_best_only=True)

In [ ]:
# As the data in hand is less as compared to the task so ImageDataGenerator is good to go.
train_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.15,
    zoom_range=0.15,
    horizontal_flip=True,
)
train_datagen.fit(X_train)

**Plot a signle sample with its augmented versions**

In [ ]:
# # select one image from the training data
# sample_image = X_train[0]

# # reshape the image to have a batch size of 1
# sample_image = sample_image.reshape((1,) + sample_image.shape)
# # sample_image.shape

# # generate batches of augmented images
# i = 0
# for X_batch, y_batch in train_datagen.flow(sample_image,y_train[0:1], batch_size=1):
#     plt.figure(i)
#     plt.imshow(X_batch[0].reshape(img_width, img_height),cmap=('gray'))
#     i += 1
#     if i > 9:
#         break

# plt.show()
# # # This will generate the augmented images using the parameters specified in the ImageDataGenerator and plot the original image as well as its augmented versions. The number of augmented images to plot is determined by the number of iterations in the for loop (in this case, it is 5). You can increase or decrease the number of iterations to generate more or fewer augmented images.






In [ ]:
# # select one image from the training data
# sample_image = X_train[0]

# # reshape the image to have a batch size of 1
# sample_image = sample_image.reshape((1,) + sample_image.shape)

# fig, axs = plt.subplots(5, figsize=(5, 20))

# # generate batches of augmented images
# for i, X_batch in enumerate(train_datagen.flow(sample_image, batch_size=1)):
#     if i >= 5:
#         break
#     axs[i].imshow(X_batch[0].reshape(img_width,img_height),cmap='gray')

# plt.show()





In [ ]:
# import matplotlib.pyplot as plt

# # Pick a single image from the dataset
# image = X_train[6]

# # Generate augmented images
# for X_batch, y_batch in train_datagen.flow(image.reshape(1,img_width, img_height,1),y_train[0:1],batch_size=1):
#     # Plot the original image
#     plt.subplot(121)
#     plt.imshow(image.reshape(img_width, img_height), cmap=('gray'))
#     plt.title('Original')
#     # Plot the augmented image
#     plt.subplot(122)
#     plt.imshow(X_batch[0].reshape(img_width, img_height), cmap=('gray'))
#     plt.title('Augmented')
#     # Show the plot
#     plt.show()
#     break

In [ ]:
# # import matplotlib.pyplot as plt

# # Generate a batch of augmented images
# for X_batch, y_batch in train_datagen.flow(X_train, y_train, batch_size=9):
#     # Create a grid of 3x3 images
#     plt.figure(figsize=(8,8))
#     for i in range(0, 9):
#         plt.subplot(330 + 1 + i)
#         plt.imshow(X_batch[i].reshape(img_width, img_height), cmap=plt.get_cmap('gray'))
#     # Show the plot
#     plt.show()
#     break
# # This code will show a grid of 3x3 augmented images, generated from the X_train dataset, and will break after one iteration.
# # Note that you will need to adjust the img_width, img_height, and channels variables to match the shape of your images.






### Augmentation + HP-Tune

In [ ]:
# #train with augmentation + hp tune
# batch_size = 32 #batch size of 32 performs the best.
# # epochs = 100


# history = model.fit_generator(
#     train_datagen.flow(X_train, y_train, batch_size=batch_size),
#     validation_data=(X_valid, y_valid),
#     steps_per_epoch=len(X_train) / batch_size,
#     epochs=epochs,
#     callbacks=[lr_reducer, tensorboard, early_stopper, checkpointer],
#     use_multiprocessing=True
# )

### Augmentation

In [ ]:
#train with augmentation
batch_size = 32 #batch size of 32 performs the best.
epochs = 100


history = model.fit_generator(
    train_datagen.flow(X_train, y_train, batch_size=batch_size),
    validation_data=(X_valid, y_valid),
    steps_per_epoch=len(X_train) / batch_size,
    epochs=epochs,
    callbacks=[tensorboard, early_stopper, checkpointer],
    use_multiprocessing=True
)

### Without Augmentation

In [ ]:
# #train without augmentation 
# history= model.fit(np.array(X_train), np.array(y_train),
#           batch_size=32,
#           steps_per_epoch=len(X_train) / batch_size,
#           epochs=epochs,
#           verbose=1,
#           validation_data=(np.array(X_valid), np.array(y_valid)),
#           shuffle=True,
#           callbacks=[tensorboard, early_stopper, checkpointer])

### Without Augmentation + HP Tuning

In [ ]:
# #train without augmentation + HP Tuning
# history= model.fit(np.array(X_train), np.array(y_train),
#           batch_size=32,
#           steps_per_epoch=len(X_train) / batch_size,
#           epochs=epochs,
#           verbose=1,
#           validation_data=(np.array(X_valid), np.array(y_valid)),
#           shuffle=True,
#           callbacks=[lr_reducer, tensorboard, early_stopper, checkpointer])

In [ ]:

# history.epoch

In [ ]:
sns.set()
fig, axes = pyplot.subplots(nrows=1, ncols=2, figsize=(12, 4))

axes[0].plot(history.epoch, history.history['accuracy'], label='train')
axes[0].plot(history.epoch, history.history['val_accuracy'], label='valid')
axes[0].set_title('Accuracy')
axes[0].legend()
axes[1].plot(history.epoch, history.history['loss'], label='train')
axes[1].plot(history.epoch, history.history['val_loss'], label='valid')
axes[1].set_title('Loss')
axes[1].legend()

pyplot.tight_layout()
pyplot.savefig('epoch_history_dcnn.png')
pyplot.show()


#  Evaluation

## Confusion Matrix on the test set

In [ ]:
predict_x = model.predict(X_test)
classes_x = np.argmax(predict_x,axis= 1)
skplt.metrics.plot_confusion_matrix(np.argmax(y_test, axis=1), classes_x,figsize=(7,7))
pyplot.savefig("confusion_matrix_dcnn.png")

print(f'total wrong test predictions: {np.sum(np.argmax(y_test, axis=1) != classes_x)}\n\n')
print(classification_report(np.argmax(y_test, axis=1), classes_x))

In [ ]:
import scikitplot as skplt
import numpy as np
import matplotlib.pyplot as plt

# Define emotion labels
emotion_labels = {0:'anger', 1:'disgust', 2:'fear', 3:'happiness', 4: 'sadness', 5: 'surprise', 6: 'neutral'}

# Make predictions
predict_x = model.predict(X_test)
classes_x = np.argmax(predict_x, axis=1)

# Plot confusion matrix
skplt.metrics.plot_confusion_matrix(np.argmax(y_test, axis=1), classes_x, figsize=(7, 7),
                                    title="Confusion Matrix (Emotion Labels)", 
                                    text_fontsize='large', cmap='Blues')

# Replace x and y tick labels with emotion names
tick_marks = np.arange(len(emotion_labels))
plt.xticks(tick_marks, list(emotion_labels.values()), rotation=45, fontsize=12)
plt.yticks(tick_marks, list(emotion_labels.values()), fontsize=12)

plt.tight_layout()
plt.show()


In [ ]:
from sklearn.metrics import confusion_matrix

y_true = np.argmax(y_test, axis=1)
cm = confusion_matrix(y_true, classes_x)

correct_pred_per_class = np.diag(cm)
incorrect_pred_per_class = np.sum(cm, axis=1) - np.diag(cm)

print("Correct predictions per class:", correct_pred_per_class)
print("Incorrect predictions per class:", incorrect_pred_per_class)


In [ ]:
np.sum(correct_pred_per_class) + np.sum(incorrect_pred_per_class)

In [ ]:
mapper = {
    0: "angry",
    1: "disgust",
    2: "fear",
    3: "happy",
    4: "sad",
    5: "suprise",
    6: "neutral"
}
np.random.seed(2)
random_dis_imgs = np.random.choice(np.where(y_valid[:, 1]==1)[0], size=9)
random_fear_imgs = np.random.choice(np.where(y_valid[:, 2]==1)[0], size=9)
fig = pyplot.figure(1, (18, 4))
for i, ( disidx, fearidx ) in enumerate(zip(random_dis_imgs, random_fear_imgs)):
        ax = pyplot.subplot(2, 9, i+1)
        sample_img = X_valid[disidx,:,:,0]
        ax.imshow(sample_img, cmap='gray')
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_title(f"true:dis, pr:{mapper[np.argmax(model.predict(sample_img.reshape(1,48,48,1))[0])]}")

       

        ax = pyplot.subplot(2, 9, i+10)
        sample_img = X_valid[fearidx,:,:,0]
        ax.imshow(sample_img, cmap='gray')
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_title(f"true:fear, pr:{mapper[np.argmax(model.predict(sample_img.reshape(1,48,48,1))[0])]}")



        pyplot.tight_layout()